In [1]:
from compare_categ import preprocess, read_csv, compare

In [2]:
from collections import defaultdict
import csv 
from csv import reader
from scipy.optimize import linear_sum_assignment
#from PIL import Image as im
from IPython.core.display import Image, display
from IPython.display import clear_output
import requests
from io import BytesIO
import json 
import pickle as pkl 


<ipython-input-2-d71306b12d10>:6: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import Image, display


In [3]:
def write(path, data):
    with open(path, 'wb') as f:
        pkl.dump(data, f)

def read(path):
    with open(path, 'rb') as f:
        return pkl.load(f)

In [17]:
csv1 = "categ/csv_input-Batch_1886_results.csv"
csv2 = "categ/csv_input-Batch_1912_results.csv"

disagree_rows = compare(csv1, csv2)

corrected = []
remaining = []
corrected_path = "categ/corrected.pkl"
remaining_path = "categ/remaining.pkl"

try:
    corrected = read(corrected_path)
    remaining = read(remaining_path)
except FileNotFoundError:
    pass 


for row1, row2 in disagree_rows:
    ans1 = preprocess(row1['Answer.skip_reason']) 
    ann1 = row1['Turkle.Username']
    ans2 = preprocess(row2['Answer.skip_reason'])
    ann2 = row2['Turkle.Username']
     
    groups = []
    new_questions = json.loads(row1['Answer.answer_questions'])
    for i, g in enumerate(json.loads(row1['Answer.answer_groups'])):
        group = []
        new_question = new_questions[i]
        for x in g:
            group.append(x['content'])
        groups.append({new_question: group})
    print(f"question: {row1['Input.questionStr']}")
    print(f"groups: {groups}")

    print(f"{ann1} said: {ans1}")
    print(f"{ann2} said: {ans2}")

    image_url = row1['Input.imgUrl'].strip('"')
    display(Image(url=image_url))

    ans = input("Which ann is correct? 1, 2, or 3 (neither)")
    ans = ans.strip()
    if ans == "1":
        corrected.append(row1)
    elif ans == "2": 
        corrected.append(row2)
    elif ans == "":
        remaining.append((row1, row2))
    else: 
        edited_row = row1.copy()
        edited_row['Ansswer.skip_reason'] = ans
        corrected.append(edited_row)

    write(corrected_path, corrected)
    write(remaining_path, remaining)
    clear_output(wait=True)


question: "Which way is the traffic heading?"
groups: [{'In which way is the traffic heading?': ['to right']}, {'Which way is the traffic heading gepgraphically?': ['west']}]
jgualla1 said: {'A/D', 'U'}
esteng said: {'A/D'}


In [5]:
corrected = read("categ/corrected.pkl")

print(len(corrected))

145


In [6]:
csv1 = "categ/csv_input-Batch_1886_results.csv"
csv2 = "categ/csv_input-Batch_1912_results.csv"

disagree_rows = compare(csv1, csv2)

agreed on 51.19% of cases (151/295)


In [7]:
import copy 
data1, data2 = read_csv(csv1), read_csv(csv2)

corrected_by_qid = {r['Input.questionId']: r for r in corrected}

clean_csv_data = copy.deepcopy(data1)
for i in range(len(data1)):
    r1 = data1[i]
    r2 = data2[i]
    clean_r = clean_csv_data[i]

    clean_tup = []
    if r1['Input.questionId'] in corrected_by_qid.keys():
        clean_tup.append(json.loads(corrected_by_qid[r1['Input.questionId']]['Answer.skip_reason']))
    else:
        clean_tup.append(r1['Answer.skip_reason'])
    clean_tup.append(json.loads(r1['Answer.skip_reason']))
    clean_tup.append(json.loads(r2['Answer.skip_reason']))
    clean_r['Answer.skip_reason'] = json.dumps(clean_tup)

import csv 
with open("categ/csv_clean.csv", "w") as f1:
    writer = csv.DictWriter(f1, fieldnames=clean_csv_data[0].keys())
    writer.writeheader()
    writer.writerows(clean_csv_data)


144


In [9]:

corrected[0]['Answer.skip_reason']

'"A/M.A/G"'